In [1]:
import csv
import pandas as pd
import numpy as np
import math
import pickle
import networkx as nx
import pickle
from collections import defaultdict

In [2]:
NUM_ROWS = 20000
following = defaultdict(list)

In [3]:
# with open('data/train.txt', 'r') as f:
#     for i in range(NUM_ROWS): 
#         if f:
#             line = next(f)
#             nodes = line.split('\t')
#             source = int(nodes[0])
#             sinks = list(map(int, nodes[1:]))
            
#             # disregard those followed by too many people
#             if len(sinks) < 2500:
#                 following[source] = sinks 
#     f.close()

In [ ]:
# NUM_ROWS = 20000
# following = defaultdict(list)

# with open('data/train.txt', 'r') as f:
#     for i in range(NUM_ROWS): 
#         if f:
#             line = next(f)
#             nodes = line.split('\t')
#             source = int(nodes[0])
#             sinks = list(map(int, nodes[1:]))
          
#             following[source] = sinks 
#     f.close()

# from itertools import product

# full_graph = nx.DiGraph()
# nodes = list(following.keys())

# for n in nodes:
#     targets = following[n]
#     combinations = list(product([n], targets))
#     full_graph.add_edges_from(combinations)
    
# nx.write_edgelist(full_graph, 'full_edgelist', data=False)

In [4]:
# from itertools import product

# full_graph = nx.DiGraph()
# nodes = list(following.keys())

# for n in nodes:
#     targets = following[n]
#     combinations = list(product([n], targets))
#     full_graph.add_edges_from(combinations)

In [2]:
train_graph = nx.read_edgelist('filtered_edgelist', create_using=nx.DiGraph, nodetype=int)

In [8]:
import csv

test_pairs = {}
with open('data/test-public.txt', 'r') as f:
    test = csv.DictReader(f, delimiter='\t')
    for row in test:
        source = row['Source']
        sink = row['Sink']
        test_pairs[source] = sink

In [4]:
print(nx.info(train_graph))

Name: 
Type: DiGraph
Number of nodes: 1364537
Number of edges: 5999507
Average in degree:   4.3967
Average out degree:   4.3967


In [ ]:
# nx.write_edgelist(full_graph, 'filtered_edgelist', data=False)

In [6]:
from random import sample

NUM_SAMPLE = 300000

# sample NUM_SAMPLE edges
# true_edges = sample(train_graph.edges, NUM_SAMPLE)

In [11]:
source_nodes = list(test_pairs.keys())

In [12]:
from tqdm import tqdm
# NEW WAY OF SAMPLING
# Sample on average 150 edges per source node
PER_SOURCE = 150

true_edges = []
carry_over = 0
for s in tqdm(source_nodes):
    candidates = list(train_graph.edges([int(s)]))
    num_need = PER_SOURCE + carry_over
    if len(candidates) >= num_need:
        true_edges.extend(sample(candidates, num_need))
        carry_over = 0
    else:
        true_edges.extend(candidates)
        carry_over = num_need - len(candidates)

100%|██████████| 2000/2000 [00:00<00:00, 5306.93it/s]


In [14]:
false_edges = []
count_edge = 0

In [16]:
from random import choice
while count_edge < NUM_SAMPLE:
    # get a random node to start
    source = int(choice(source_nodes))
    # find missing edges from 2 distance
    one_neighbors = list(train_graph.neighbors(source))
    count_n1 = 0
    for n1 in one_neighbors:
        # get up to 150 missing edges from source
        if count_n1 > 150:
            count_edge += count_n1
            break
        else:
            count_n2 = 0
            two_neighbors = list(train_graph.neighbors(n1))
        # get up to 15 missing edges from two neighbors
        for n2 in two_neighbors:
            if count_n2 > 15:
                count_n1 += count_n2
                break
            else:
                count_n3 = 0
                # check three neighbors
                three_neighbors = list(train_graph.neighbors(n2))
                
                # get up to 5 missing edges from three neighbors
                for n3 in three_neighbors:
                    if count_n3 > 5:
                        count_n2 += count_n3
                        break
                    else:
                        if not train_graph.has_edge(source, n3):
                            count_n3 += 1
                            false_edges.append((source, n3))
                
                # done with three neighbors, go back to current n2
                if not train_graph.has_edge(source, n2):
                    count_n2 += 1
                    false_edges.append((source, n2))

In [18]:
def assign_label(pair, graph):
    u, v = pair[0], pair[1]
    return (int(graph.has_edge(u, v)))

def concatenate(node_set, label):
    dataset = pd.DataFrame({'nodes': node_set, 'label': label})
    return dataset

from functools import partial

In [19]:
# Label creation
y_1 = list(map(partial(assign_label, graph=train_graph), true_edges))
y_0 = list(map(partial(assign_label, graph=train_graph), false_edges))

In [20]:
# append y_true to y_train & true_edges to train_small
y_train = y_1 + y_0
edges_train = true_edges + false_edges

In [21]:
train_df = concatenate(edges_train, y_train)

In [22]:
train_df.to_csv("600k_train.csv", index=False)